# NerualNet
활성화 함수

$$h(x) = \begin{cases} 0 \; (x \leq \;0)\\ 1 \; (x > \; 0) \end{cases}$$

활성화 함수는 임계 값을 경계로 출력값을 결정함.

퍼셉트론은 계단함수 **step function** 을 사용함
$$ a \;=\; b + w_1 x_1 + w_2 x_2\\  
 y \;=\; h(a) $$

계단함수와 sigmoid 함수 모두 **비선형함수**  

NeuralNet에서 비선형함수를 사용해야하는 이유는 선형함수로 하면, 신경망의 층을 깊게 하는 의미가 없어지기 때문.  

## 1. step_function 구현하기

In [4]:
def step_funcion(x):
    if x > 0:
        return 1
    else:
        return 0

위 코드는 배열을 받을 수 없음.  
실수만 받을 수 있음.

In [3]:
def step_function(x):
    y=x>0
    return y.astype(int)

    """
    def step_function(x):
        return np.array(x>0, dtype=int)
    """

## 2. sigmoid 함수 구현하기

$$ h(x) = \frac {1}{1+ exp(-x)}$$

In [9]:
import numpy as np
def sigmoid(x):
    return 1/(1+np.exp(-x))

## 3. ReLU
Rectified Linear Unit

입력이 0을 넘으면 그 입력을 그대로 출력하고, 0이하면 0을 출력

In [13]:
import numpy as np
def relu(x):
    return np.maximum(0,x)

## 4. 3층 신경망 구현

출력층의 활성화 함수는 풀고자하는 문제에 맞게 설정한다.

회귀 - 항등함수

분류 - Softmax (시그모이드)

In [ ]:
#입력층에서 1층으로의 전달
import numpy as np
X = np.array([1.0,0.5])
W1 = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
B1 = np.array([0.1,0.2,0.3])

print(W1.shape) #(2,3)
print(X.shape) #(2,)
print(B1.shape) #(3,)

A1 = np.dot(X,W1) +B1 # matrix

(2, 3)
(2,)
(3,)


In [14]:
Z1 = sigmoid(A1) # activation function

print(A1)
print(Z1)

[0.3 0.7 1.1]
[0.57444252 0.66818777 0.75026011]


In [15]:
# 1층에서 2층으으로 가는 과정
W2 = np.array([[0.1,0.4],[0.2,0.5],[0.3,0.6]])
B2 = np.array([0.1,0.2])

print(Z1.shape) #(3,) # 1층의 뉴런 수 # 활성화 함수 통과
print(W2.shape) #(3,2)
print(B2.shape) #(2,)

A2 = np.dot(Z1,W2) + B2
Z2 = sigmoid(A2)

(3,)
(3, 2)
(2,)


항등함수인 identity_function을 정의하고, 이를 출력층의 활성화 함수로 이용.   
항등함수는 입력을 그대로 출력하는 함수.  
출력층의 활성화 함수는 은닉층의 활성화 함수랑 다름.

In [16]:
# 2층에서 출력층으로 가는 과정
def identity_function(x):
    return x

W3 = np.array([[0.1,0.3],[0.2,0.4]])
B3 = np.array([0.1,0.2])

A3 = np.dot(Z2,W3)+ B3
Y = identity_function(A3) # Y=A3

**Note**   
> 출력층의 활성화 함수는 풀고자 하는 문제의 성질에 맞게 정의   
> * 회귀에는 항등   
> * 2클래스 분류 sigmoid   
> * 다중클래스 분류 softmax   

## 5. 구현 정리

In [17]:
def init__network():
    network = {}
    network['W1'] = np.array([[0.1,0.3,0.5],[0.2,0.4,0.6]])
    network['b1'] = np.array([0.1,0.2,0.3])
    network['W2'] = np.array([[0.1,0.4],[0.2,0.5],[0.3,0.6]])
    network['b2'] = np.array([0.1,0.2])
    network['W3'] = np.array([[0.1,0.3],[0.2,0.4]])
    network['b3'] = np.array([0.1,0.2])
    
    return network

def forward(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x,W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1,W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2,W3) + b3
    y = identity_function(a3)
    
    return y

network = init__network()
x = np.array([1.0,0.5])
y = forward(network, x)
print(y)

[0.31682708 0.69627909]


## 6. Softmax

softmax()는 0에서 1.0 사이의 실수  
softmax의 출력의 총합은 1  
따라서 확률로 해석 가능 

$$ y_k \;=\; \frac{exp(a_k)}{\displaystyle\sum_{i=1}^{n}{exp(a_i)}}  $$

In [18]:
def sofmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

softmax는 overflow가 있을 수 있음.

In [19]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c) # 오버플로우 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

softmax 함수를 적용해도 각 원소의 대소 관계는 변하지 않음.
y = exp(x)가 단조 증가 함수이기 때문  
* 따라서 가장 큰 뉴런의 위치는 달라지지 않는다.

추론단계에서 사용 **X**   
학습단계의 출력층에서 사용

## 7. 추론과정 구현

추론과정은 forward propagation이라고 함. (순전파)  
* 1. 먼저 훈련데이터를 사용해 가중치 매개변수를 학습하고,
* 2. 추론단계에서 앞서 학습한 매개변수를 사용하여 입력 데이터를 분류합니다.

In [ ]:
x,t = get_data()
network=init__network()

accuracy_cnt=0
for i in range(len(x)):
    y=forward(network,x[i])
    p=np.argmax(y) # 확률이 가장 높은 원소의 인덱스를 얻는다.
    if p==t[i]:
        accuracy_cnt+=1
print("Accuracy:"+str(float(accuracy_cnt)/len(x)))

### 배치처리

In [ ]:
x,t = get_data()
network = init_network()
batch_size = 100 # 배치 크기
accuracy_cnt = 0

for i in range(0,len(x), batch_size): # start=0, end=len(x), step=batch_size
    x_batch = x[i:i+batch_size] # 입력 데이터를 묶는거임.
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1) # 각 데이터의 확률이 가장 높은 원소의 인덱스를 얻는다. 
    # axis=1은 행 방향( 100x10 배열중 1번째 차원을 구성하는 각 원소에서 (1번째 차원을 축으로)최댓값의 인덱스를 찾도록 함)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])
    
print("Accuracy:"+str(float(accuracy_cnt)/len(x)))